# Malicious URL detection using Kolmogorov Complexity

## By Davide Garcia Civiero - MS IA

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import deflate
import time
from Levenshtein import distance as lev

In [2]:
df = pd.read_csv('malicious_phish.csv', sep = ',')

df.head()

,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement


### Création des datasets d'entraînement et de test

In [3]:
df_train = df[0:7500]
df_test = df[7500:10000]

In [4]:
df_train

,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement
...,...,...
7495,nndb.com/people/451/000032355/,benign
7496,superpages.com/bp/Shawnee-KS/Shawnee-Mission-F...,benign
7497,youtube.com/watch?v=wgkaPrAI660,benign
7498,kumc.edu/gec/support/hemophil.html,benign


In [5]:
df_test

,url,type
7500,http://alfalfalfa.com/tag/%E3%83%9D%E3%83%86%E...,benign
7501,http://www.tandemimmobilier.fr/index.php?optio...,defacement
7502,ted.com/talks/mike_rowe_celebrates_dirty_jobs....,benign
7503,potholesinmyblog.com/handsome-furs-sound-kapital/,benign
7504,catholicculture.org/culture/reviews/view.cfm?r...,benign
...,...,...
9995,movies.tvguide.com/eastern-promises/285629,benign
9996,http://themeforest.net/item/moksha-one-multi-p...,benign
9997,flaggedrevs.labs.wikimedia.org/wiki/New_York_S...,benign
9998,thetakeaway.org/people/victor-begg/,benign


In [6]:
print("Le dataset d'entraînement contient {} URLs".format(len(df_train)))
print("Le dataset de test contient {} URLs".format(len(df_test)))

Le dataset d'entraînement contient 7500 URLs
Le dataset de test contient 2500 URLs


### Division du dataset en deux "sous-datasets" : malicieux et bénin

In [7]:
# Stockage des URL malicieux et bénins du dataset train
data_m = list()  # dataset malicieux
data_b = list()  # dataset bénin
data_train = list()  # liste contenant tous les URL d'entraînement

label_train_true = list()  # liste avec les vrais labels

for idx, row in df_train.iterrows():
    if row[1] == 'benign':
        data_b.append(row[0])
        label_train_true.append(1)
    else:
        data_m.append(row[0])
        label_train_true.append(-1)
        
    data_train.append(row[0])

In [8]:
# On récupère les labels des URL du dataset TEST
label_test_true = list()
data_test = list()  # dataset de test

for idx, row in df_test.iterrows():
    if row[1] == 'benign':
        label_test_true.append(1)
    else:
        label_test_true.append(-1)
    data_test.append(row[0])

In [9]:
print("Il y a {} URLs bénins".format(len(data_b)))
print("Il y a {} URLs malicieux".format(len(data_m)))

Il y a 5498 URLs bénins
Il y a 2002 URLs malicieux


## Implémentation de l'algorithme de détection d'URL malicieux

In [10]:
def predict(dataset):
    # Concatene tous les éléments des deux datasets
    d_m = ''.join(data_m)
    d_b = ''.join(data_b)
    
    # liste des labels des urls (-1 = malicieux ou 1 = bénin)
    url_labels = list()

    # Boucle sur tous les éléments du dataset
    for url in dataset:
        # Concaténer d_m et s, puis d_b et s
        d_ms = d_m + url
        d_bs = d_b + url

        # Compression de d_ms et d_bs
        comp_dms = deflate.gzip_compress(bytes(d_ms, 'utf-8'))
        comp_dbs = deflate.gzip_compress(bytes(d_bs, 'utf-8'))

        comp_dm = deflate.gzip_compress(bytes(d_m, 'utf-8'))
        comp_db = deflate.gzip_compress(bytes(d_b, 'utf-8'))

        # Longueur des compressions
        g_dms = len(comp_dms)
        g_dbs = len(comp_dbs)

        # Calcul du ratio m
        g_dm = len(comp_dm)
        g_db = len(comp_db)

        g_s_dm = g_dms - g_dm  # g(s|dm)
        g_s_db = g_dbs - g_db  # g(s|db)

        m_ratio = (g_s_dm - g_s_db) / (g_s_dm + g_s_db)

        # Classification
        if m_ratio < 0:
            url_labels.append(-1)
        else:
            url_labels.append(1)
            
    return url_labels

In [11]:
def accuracy(labels_true, labels_pred):
    counter = 0
    for i in range(len(labels_pred)):
        if labels_true[i] == labels_pred[i]:
            counter += 1
    print("Le taux de précision est de {}%".format(counter/len(labels_pred) * 100))
    
    return

### Résultats sur le dataset train

In [12]:
start_time = time.time()

url_labels_train = predict(data_train)
accuracy(label_train_true, url_labels_train)

print("Temps d'exécution {} secondes".format(round(time.time() - start_time, 3)))

Le taux de précision est de 93.37333333333333%
Temps d'exécution 120.876 secondes


### Résultats sur le dataset test

In [13]:
start_time = time.time()

url_labels_test = predict(data_test)
accuracy(label_test_true, url_labels_test)

print("Temps d'exécution {} secondes".format(round(time.time() - start_time, 3)))

Le taux de précision est de 90.88000000000001%
Temps d'exécution 42.071 secondes


## Application de la distance de Levenshtein

In [14]:
# Constitution des datasets d'entraînement et de test

df_train = df[0:7500]
df_test = df[7500:10000]

# Stockage des URL malicieux et bénins du dataset train
data_m = list()  # dataset malicieux
data_b = list()  # dataset bénin
data_train = list()  # liste contenant tous les URL d'entraînement

label_train_true = list()  # liste avec les vrais labels

for idx, row in df_train.iterrows():
    if row[1] == 'benign':
        data_b.append(row[0])
        label_train_true.append(1)
    else:
        data_m.append(row[0])
        label_train_true.append(-1)
        
    data_train.append(row[0])

# On récupère les labels des URL du dataset TEST
label_test_true = list()
data_test = list()  # dataset de test

for idx, row in df_test.iterrows():
    if row[1] == 'benign':
        label_test_true.append(1)
    else:
        label_test_true.append(-1)
    data_test.append(row[0])

In [15]:
print("Le dataset d'entraînement contient {} URLs".format(len(df_train)))
print("Le dataset de test contient {} URLs".format(len(df_test)))

Le dataset d'entraînement contient 7500 URLs
Le dataset de test contient 2500 URLs


In [16]:
def predict_lev(dataset):
    # Concatene tous les éléments des deux datasets
    d_m = ''.join(data_m)
    d_b = ''.join(data_b)
    
    # liste des labels des urls (-1 = malicieux ou 1 = bénin)
    url_labels = list()

    # Boucle sur tous les éléments du dataset
    for url in dataset:
        # Concaténer d_m et s, puis d_b et s
        d_ms = d_m + url
        d_bs = d_b + url
        
        # Distance de Levenshtein entre DMS et DM / DBS et DB
        g_s_dm = lev(d_ms, d_m)
        g_s_db = lev(d_bs, d_b)
        m_ratio = (g_s_dm - g_s_db) / (g_s_dm + g_s_db)

        # Classification
        if m_ratio < 0:
            url_labels.append(-1)
        else:
            url_labels.append(1)
            
    return url_labels

In [17]:
start_time = time.time()

url_labels_train = predict_lev(data_train)
accuracy(label_train_true, url_labels_train)

print("Temps d'exécution {} secondes".format(round(time.time() - start_time, 3)))

Le taux de précision est de 73.30666666666666%
Temps d'exécution 2.521 secondes


In [18]:
start_time = time.time()

url_labels_test = predict_lev(data_test)
accuracy(label_test_true, url_labels_test)

print("Temps d'exécution {} secondes".format(round(time.time() - start_time, 3)))

Le taux de précision est de 73.08%
Temps d'exécution 0.879 secondes
